In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

### **Data Reading**

In [0]:
df = spark.read.format("parquet") \
    .load("abfss://bronze@dlsdatabrickseteteamea.dfs.core.windows.net/customers")


In [0]:
df.display()

In [0]:
df = df.select(["customer_id", "first_name", "last_name", "email", "city", "state"])
# df = df.drop("_rescued_data")

In [0]:
df.display()

In [0]:
# df = df.withColumn("domain", split(col("email"), "@")[1])
df = df.withColumn("domains", split(col("email"), "@").getItem(1))
df.display()

In [0]:
df.groupBy("domains").agg(count("customer_id").alias("total_customers")).sort("total_customers", ascending=False).display()
# ถ้าไม่ได้ assign ค่าให้ตัวแปร ก็เหมือนกับแค่แสดดงผลเฉยๆ

In [0]:
# df_gmail = df.filter(col('domains').like('%gmail.com'))

df_gmail = df.filter(col('domains')=="gmail.com")
# print("gmail.com = ",df_gmail.count())
df_gmail.display()
# time.sleep(5) # 5 seconds

df_yahoo = df.filter(col('domains')=="yahoo.com")
# print("yahoo.com = ",df_yahoo.count())
df_yahoo.display()
# time.sleep(5)

df_hotmail = df.filter(col('domains')=="hotmail.com")
# print("hotmail.com = ",df_hotmail.count())
df_hotmail.display()
# time.sleep(5)

In [0]:
df = df.withColumn("full_name", concat(col("first_name"), lit(" "), col("last_name")))
df = df.drop("first_name", "last_name")
df.display()

In [0]:
df.write.format("delta").mode("append").save("abfss://silver@dlsdatabrickseteteamea.dfs.core.windows.net/customers")

### **Create The Delta Table from External Delta file**

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cata.silver.customers_silver
USING DELTA
LOCATION 'abfss://silver@dlsdatabrickseteteamea.dfs.core.windows.net/customers'